# ML Model Training and Persistence - Prototyping Notebook

This notebook is part of **Story 2.1: ML Model Training and Persistence**.

Goals:
- Load football match data from SQLite.
- Preprocess features.
- Train a classifier on match outcomes.
- Save the trained model for inference.

Steps:
1. Setup and Data Loading with Feature Engineering (SQL-based)
2. Data Exploration
3. Data Preprocessing
4. Model Training, Evaluation, and Persistence

## 1. Setup and Data Loading with Feature Engineering (SQL-based)

In [ ]:
import sqlite3
import pandas as pd
import os
from pandas.io.sql import DatabaseError

db_path = '../football.db'

if not os.path.exists(db_path):
    raise FileNotFoundError(f"Database '{db_path}' not found. Run db_setup.py first.")

query = """
WITH
  team_games AS (
    SELECT
      rowid, Date, HomeTeam AS team, 'home' as side,
      CASE FTR WHEN 'H' THEN 3 WHEN 'D' THEN 1 ELSE 0 END AS points,
      HS AS shots, HC AS corners, HF AS fouls,
      FTHG AS goals_scored,
      FTAG AS goals_conceded
    FROM matches
    UNION ALL
    SELECT
      rowid, Date, AwayTeam AS team, 'away' as side,
      CASE FTR WHEN 'A' THEN 3 WHEN 'D' THEN 1 ELSE 0 END AS points,
      AS AS shots, AC AS corners, AF AS fouls,
      FTAG AS goals_scored,
      FTHG AS goals_conceded
    FROM matches
  ),
  lagged AS (
    SELECT
      rowid, team, side, Date,
      LAG(points, 1) OVER (PARTITION BY team ORDER BY Date) AS points_1,
      LAG(points, 2) OVER (PARTITION BY team ORDER BY Date) AS points_2,
      LAG(points, 3) OVER (PARTITION BY team ORDER BY Date) AS points_3,
      LAG(points, 4) OVER (PARTITION BY team ORDER BY Date) AS points_4,
      LAG(points, 5) OVER (PARTITION BY team ORDER BY Date) AS points_5,
      LAG(points, 6) OVER (PARTITION BY team ORDER BY Date) AS points_6,
      LAG(shots, 1) OVER (PARTITION BY team ORDER BY Date) AS shots_1,
      LAG(shots, 2) OVER (PARTITION BY team ORDER BY Date) AS shots_2,
      LAG(shots, 3) OVER (PARTITION BY team ORDER BY Date) AS shots_3,
      LAG(shots, 4) OVER (PARTITION BY team ORDER BY Date) AS shots_4,
      LAG(shots, 5) OVER (PARTITION BY team ORDER BY Date) AS shots_5,
      LAG(shots, 6) OVER (PARTITION BY team ORDER BY Date) AS shots_6,
      LAG(corners, 1) OVER (PARTITION BY team ORDER BY Date) AS corners_1,
      LAG(corners, 2) OVER (PARTITION BY team ORDER BY Date) AS corners_2,
      LAG(corners, 3) OVER (PARTITION BY team ORDER BY Date) AS corners_3,
      LAG(corners, 4) OVER (PARTITION BY team ORDER BY Date) AS corners_4,
      LAG(corners, 5) OVER (PARTITION BY team ORDER BY Date) AS corners_5,
      LAG(corners, 6) OVER (PARTITION BY team ORDER BY Date) AS corners_6,
      LAG(fouls, 1) OVER (PARTITION BY team ORDER BY Date) AS fouls_1,
      LAG(fouls, 2) OVER (PARTITION BY team ORDER BY Date) AS fouls_2,
      LAG(fouls, 3) OVER (PARTITION BY team ORDER BY Date) AS fouls_3,
      LAG(fouls, 4) OVER (PARTITION BY team ORDER BY Date) AS fouls_4,
      LAG(fouls, 5) OVER (PARTITION BY team ORDER BY Date) AS fouls_5,
      LAG(fouls, 6) OVER (PARTITION BY team ORDER BY Date) AS fouls_6,
      LAG(goals_scored, 1) OVER (PARTITION BY team ORDER BY Date) AS goals_scored_1,
      LAG(goals_scored, 2) OVER (PARTITION BY team ORDER BY Date) AS goals_scored_2,
      LAG(goals_scored, 3) OVER (PARTITION BY team ORDER BY Date) AS goals_scored_3,
      LAG(goals_scored, 4) OVER (PARTITION BY team ORDER BY Date) AS goals_scored_4,
      LAG(goals_scored, 5) OVER (PARTITION BY team ORDER BY Date) AS goals_scored_5,
      LAG(goals_scored, 6) OVER (PARTITION BY team ORDER BY Date) AS goals_scored_6,
      LAG(goals_conceded, 1) OVER (PARTITION BY team ORDER BY Date) AS goals_conceded_1,
      LAG(goals_conceded, 2) OVER (PARTITION BY team ORDER BY Date) AS goals_conceded_2,
      LAG(goals_conceded, 3) OVER (PARTITION BY team ORDER BY Date) AS goals_conceded_3,
      LAG(goals_conceded, 4) OVER (PARTITION BY team ORDER BY Date) AS goals_conceded_4,
      LAG(goals_conceded, 5) OVER (PARTITION BY team ORDER BY Date) AS goals_conceded_5,
      LAG(goals_conceded, 6) OVER (PARTITION BY team ORDER BY Date) AS goals_conceded_6
    FROM team_games
  ),
  game_features AS (
    SELECT *,
      (
        COALESCE(points_1, 0) * 1.0 +
        COALESCE(points_2, 0) * 0.8 +
        COALESCE(points_3, 0) * 0.6 +
        COALESCE(points_4, 0) * 0.4 +
        COALESCE(points_5, 0) * 0.2 +
        COALESCE(points_6, 0) * 0.1
      ) / 9.3 * 10 AS form_score
    FROM lagged
  )
SELECT
  m.*,
  h.form_score AS HomeTeam_FormScore,
  a.form_score AS AwayTeam_FormScore,
  h.shots_1, h.shots_2, h.shots_3, h.shots_4, h.shots_5, h.shots_6,
  a.shots_1 AS a_shots_1, a.shots_2 AS a_shots_2, a.shots_3 AS a_shots_3,
  a.shots_4 AS a_shots_4, a.shots_5 AS a_shots_5, a.shots_6 AS a_shots_6,
  h.corners_1, h.corners_2, h.corners_3, h.corners_4, h.corners_5, h.corners_6,
  a.corners_1 AS a_corners_1, a.corners_2 AS a_corners_2, a.corners_3 AS a_corners_3,
  a.corners_4 AS a_corners_4, a.corners_5 AS a_corners_5, a.corners_6 AS a_corners_6,
  h.fouls_1, h.fouls_2, h.fouls_3, h.fouls_4, h.fouls_5, h.fouls_6,
  a.fouls_1 AS a_fouls_1, a.fouls_2 AS a_fouls_2, a.fouls_3 AS a_fouls_3,
  a.fouls_4 AS a_fouls_4, a.fouls_5 AS a_fouls_5, a.fouls_6 AS a_fouls_6,
  h.goals_scored_1, h.goals_scored_2, h.goals_scored_3, h.goals_scored_4, h.goals_scored_5, h.goals_scored_6,
  a.goals_scored_1 AS a_goals_scored_1, a.goals_scored_2 AS a_goals_scored_2, a.goals_scored_3 AS a_goals_scored_3,
  a.goals_scored_4 AS a_goals_scored_4, a.goals_scored_5 AS a_goals_scored_5, a.goals_scored_6 AS a_goals_scored_6,
  h.goals_conceded_1, h.goals_conceded_2, h.goals_conceded_3, h.goals_conceded_4, h.goals_conceded_5, h.goals_conceded_6,
  a.goals_conceded_1 AS a_goals_conceded_1, a.goals_conceded_2 AS a_goals_conceded_2, a.goals_conceded_3 AS a_goals_conceded_3,
  a.goals_conceded_4 AS a_goals_conceded_4, a.goals_conceded_5 AS a_goals_conceded_5, a.goals_conceded_6 AS a_goals_conceded_6
FROM matches m
JOIN game_features h ON m.rowid = h.rowid AND h.side = 'home'
JOIN game_features a ON m.rowid = a.rowid AND a.side = 'away'
"""

conn = sqlite3.connect(db_path)

try:
    df = pd.read_sql_query(query, conn)
    print(f"✅ Loaded and processed {len(df)} rows from 'matches' table with form scores.")
    
    # Calcul des moyennes en Python
    df["HomeTeam_AvgShots"] = df[["shots_1", "shots_2", "shots_3", "shots_4", "shots_5", "shots_6"]].mean(axis=1)
    df["AwayTeam_AvgShots"] = df[["a_shots_1", "a_shots_2", "a_shots_3", "a_shots_4", "a_shots_5", "a_shots_6"]].mean(axis=1)
    
    df["HomeTeam_AvgCorners"] = df[["corners_1", "corners_2", "corners_3", "corners_4", "corners_5", "corners_6"]].mean(axis=1)
    df["AwayTeam_AvgCorners"] = df[["a_corners_1", "a_corners_2", "a_corners_3", "a_corners_4", "a_corners_5", "a_corners_6"]].mean(axis=1)
    
    df["HomeTeam_AvgFouls"] = df[["fouls_1", "fouls_2", "fouls_3", "fouls_4", "fouls_5", "fouls_6"]].mean(axis=1)
    df["AwayTeam_AvgFouls"] = df[["a_fouls_1", "a_fouls_2", "a_fouls_3", "a_fouls_4", "a_fouls_5", "a_fouls_6"]].mean(axis=1)
    
    # Nouvelles moyennes de buts
    df["HomeTeam_AvgGoalsScored"] = df[["goals_scored_1", "goals_scored_2", "goals_scored_3", "goals_scored_4", "goals_scored_5", "goals_scored_6"]].mean(axis=1)
    df["AwayTeam_AvgGoalsScored"] = df[["a_goals_scored_1", "a_goals_scored_2", "a_goals_scored_3", "a_goals_scored_4", "a_goals_scored_5", "a_goals_scored_6"]].mean(axis=1)
    
    df["HomeTeam_AvgGoalsConceded"] = df[["goals_conceded_1", "goals_conceded_2", "goals_conceded_3", "goals_conceded_4", "goals_conceded_5", "goals_conceded_6"]].mean(axis=1)
    df["AwayTeam_AvgGoalsConceded"] = df[["a_goals_conceded_1", "a_goals_conceded_2", "a_goals_conceded_3", "a_goals_conceded_4", "a_goals_conceded_5", "a_goals_conceded_6"]].mean(axis=1)
    
except DatabaseError as e:
    df = pd.DataFrame()
    print(f"❌ Error loading data: {e}")
finally:
    conn.close()


## 2. Data Exploration

In [ ]:
if df.empty:
    print("❌ DataFrame is empty. Cannot proceed.")
else:
    display(df.head())
    print("\nDataFrame Info:")
    display(df.info())
    print("\nData Description:")
    display(df.describe(include='all'))
    
    # Vérification des nouvelles caractéristiques de buts
    print("\n📊 Nouvelles caractéristiques de buts:")
    goals_features = ['HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored', 'HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded']
    for feature in goals_features:
        if feature in df.columns:
            print(f"{feature}: min={df[feature].min():.2f}, max={df[feature].max():.2f}, mean={df[feature].mean():.2f}")
        else:
            print(f"❌ {feature} not found in DataFrame")

## 3. Data Preprocessing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

categorical_features = ['HomeTeam', 'AwayTeam']
# Mise à jour des caractéristiques numériques - remplacement de HS et AS par les moyennes de buts
numerical_features = [
    'HomeTeam_FormScore', 'AwayTeam_FormScore',
    'HomeTeam_AvgShots', 'AwayTeam_AvgShots',
    'HomeTeam_AvgCorners', 'AwayTeam_AvgCorners',
    'HomeTeam_AvgFouls', 'AwayTeam_AvgFouls',
    'HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored',
    'HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded'
]

for col in numerical_features:
    if col in df.columns:
        df[col] = pd.to_numeric(df[col], errors='coerce')
    else:
        print(f"⚠️ Warning: {col} not found in DataFrame")

essential_cols = [col for col in numerical_features if col in df.columns] + categorical_features + ['FTR']
df_processed = df.dropna(subset=essential_cols)

# Gestion des valeurs manquantes pour les nouvelles caractéristiques
goals_features = ['HomeTeam_AvgGoalsScored', 'AwayTeam_AvgGoalsScored', 'HomeTeam_AvgGoalsConceded', 'AwayTeam_AvgGoalsConceded']
for col in goals_features:
    if col in df_processed.columns:
        # Remplacer les NaN par la moyenne globale
        df_processed[col] = df_processed[col].fillna(df_processed[col].mean())
        print(f"✅ Handled missing values for {col}")

available_numerical_features = [col for col in numerical_features if col in df_processed.columns]
features = categorical_features + available_numerical_features
X = df_processed[features]
y = df_processed['FTR']

numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer([
    ('num', numeric_pipeline, available_numerical_features),
    ('cat', categorical_pipeline, categorical_features)
])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"✅ Data preprocessing complete. Training with {len(X_train)} samples.")
print(f"📊 Using features: {features}")


## 4. Model Training, Evaluation, and Persistence

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

model_pipeline = Pipeline([
    ('preprocess', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

model_pipeline.fit(X_train, y_train)

y_pred = model_pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"🎯 Model Accuracy: {accuracy:.2f}")
print(f"\n📊 Classification Report:")
print(classification_report(y_test, y_pred))

# Validation de l'absence de fuite de données
print("\n🔍 Validation - Caractéristiques utilisées:")
feature_names = (categorical_features + available_numerical_features)
for feature in feature_names:
    if 'HS' in feature or 'AS' in feature:
        print(f"⚠️ ATTENTION: {feature} pourrait causer une fuite de données")
    else:
        print(f"✅ {feature} - OK (données historiques)")

os.makedirs('models', exist_ok=True)
model_path = 'models/prediction_model.pkl'
joblib.dump(model_pipeline, model_path)
print(f"\n💾 Model saved to {model_path}")
